In [1]:
# Dependencies
import pandas as pd
import re

In [2]:
url = 'https://www.pro-football-reference.com/years/2007/games.htm'

In [3]:
tables = pd.read_html(url)
tables

[          Week  Day         Date     Time            Winner/tie Unnamed: 5  \
 0            1  Thu  September 6   8:39PM    Indianapolis Colts        NaN   
 1            1  Sun  September 9  12:59PM     Carolina Panthers          @   
 2            1  Sun  September 9   1:00PM     Minnesota Vikings        NaN   
 3            1  Sun  September 9   1:02PM   Washington Redskins        NaN   
 4            1  Sun  September 9   1:02PM        Denver Broncos          @   
 ..         ...  ...          ...      ...                   ...        ...   
 279   Division  Sun   January 13   1:06PM    San Diego Chargers          @   
 280   Division  Sun   January 13   4:45PM       New York Giants          @   
 281  ConfChamp  Sun   January 20   3:05PM  New England Patriots        NaN   
 282  ConfChamp  Sun   January 20   6:42PM       New York Giants          @   
 283  SuperBowl  Sun   February 3   6:30PM       New York Giants          N   
 
                 Loser/tie Unnamed: 7 PtsW PtsL Yd

In [4]:
tables[0]

,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,PtsW,PtsL,YdsW,TOW,YdsL,TOL
0,1,Thu,September 6,8:39PM,Indianapolis Colts,NaN,New Orleans Saints,boxscore,41,10,452,1,293,3
1,1,Sun,September 9,12:59PM,Carolina Panthers,@,St. Louis Rams,boxscore,27,13,385,2,238,2
2,1,Sun,September 9,1:00PM,Minnesota Vikings,NaN,Atlanta Falcons,boxscore,24,3,302,1,265,2
3,1,Sun,September 9,1:02PM,Washington Redskins,NaN,Miami Dolphins,boxscore,16,13,400,2,273,1
4,1,Sun,September 9,1:02PM,Denver Broncos,@,Buffalo Bills,boxscore,15,14,470,1,184,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Division,Sun,January 13,1:06PM,San Diego Chargers,@,Indianapolis Colts,boxscore,28,24,411,1,446,3
280,Division,Sun,January 13,4:45PM,New York Giants,@,Dallas Cowboys,boxscore,21,17,230,0,336,1
281,ConfChamp,Sun,January 20,3:05PM,New England Patriots,NaN,San Diego Chargers,boxscore,21,12,347,3,311,2
282,ConfChamp,Sun,January 20,6:42PM,New York Giants,@,Green Bay Packers,boxscore,23,20,377,1,264,2


In [5]:
df_table = tables[0]
cols = [1,3,5,7]
df_table.drop(df_table.columns[cols],axis=1,inplace=True)
df_table

,Week,Date,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL
0,1,September 6,Indianapolis Colts,New Orleans Saints,41,10,452,1,293,3
1,1,September 9,Carolina Panthers,St. Louis Rams,27,13,385,2,238,2
2,1,September 9,Minnesota Vikings,Atlanta Falcons,24,3,302,1,265,2
3,1,September 9,Washington Redskins,Miami Dolphins,16,13,400,2,273,1
4,1,September 9,Denver Broncos,Buffalo Bills,15,14,470,1,184,1
...,...,...,...,...,...,...,...,...,...,...
279,Division,January 13,San Diego Chargers,Indianapolis Colts,28,24,411,1,446,3
280,Division,January 13,New York Giants,Dallas Cowboys,21,17,230,0,336,1
281,ConfChamp,January 20,New England Patriots,San Diego Chargers,21,12,347,3,311,2
282,ConfChamp,January 20,New York Giants,Green Bay Packers,23,20,377,1,264,2


In [6]:
df_table = df_table[df_table.Week.apply(lambda x: str(x).isnumeric())]
df_table = df_table.reset_index(drop=True)
df_table

,Week,Date,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL
0,1,September 6,Indianapolis Colts,New Orleans Saints,41,10,452,1,293,3
1,1,September 9,Carolina Panthers,St. Louis Rams,27,13,385,2,238,2
2,1,September 9,Minnesota Vikings,Atlanta Falcons,24,3,302,1,265,2
3,1,September 9,Washington Redskins,Miami Dolphins,16,13,400,2,273,1
4,1,September 9,Denver Broncos,Buffalo Bills,15,14,470,1,184,1
...,...,...,...,...,...,...,...,...,...,...
251,17,December 30,San Diego Chargers,Oakland Raiders,30,17,253,1,316,4
252,17,December 30,Arizona Cardinals,St. Louis Rams,48,19,422,2,234,3
253,17,December 30,Denver Broncos,Minnesota Vikings,22,19,362,0,332,3
254,17,December 30,Baltimore Ravens,Pittsburgh Steelers,27,21,334,1,264,3


In [7]:
points_dict = {}
yards_dict = {}
tov_dict = {}
example = df_table.head()
#cond = df_table['Winner/tie'] == 'Indianapolis Colts'  
#cond2 = df_table['Loser/tie'] == 'Indianapolis Colts'
#colts = df_table[cond | cond2]

#print(colts)
win_dif = []
lose_dif = []
win_yards = []
lose_yards = []
win_tov = []
lose_tov = []

for i in df_table.iterrows():
        
        #point difference
        if i[1]['Winner/tie'] in points_dict:
            win_dif.append(points_dict[i[1]['Winner/tie']])
            points_dict[i[1]['Winner/tie']] = points_dict[i[1]['Winner/tie']] + int(i[1]['PtsW']) - int(i[1]['PtsL'])
        else:
            points_dict[i[1]['Winner/tie']] = int(i[1]['PtsW']) - int(i[1]['PtsL'])
            win_dif.append(0)
        #print(points_dict[i[1]['Winner/tie']])

        
        if i[1]['Loser/tie'] in points_dict:
            lose_dif.append(points_dict[i[1]['Loser/tie']])
            points_dict[i[1]['Loser/tie']] = points_dict[i[1]['Loser/tie']] - int(i[1]['PtsW']) + int(i[1]['PtsL'])
        else:
            points_dict[i[1]['Loser/tie']] = -int(i[1]['PtsW']) + int(i[1]['PtsL'])
            lose_dif.append(0)
            
        #net yards
        if i[1]['Winner/tie'] in yards_dict:
            win_yards.append(yards_dict[i[1]['Winner/tie']])
            yards_dict[i[1]['Winner/tie']] = yards_dict[i[1]['Winner/tie']] + int(i[1]['YdsW']) - int(i[1]['YdsL'])
        else:
            yards_dict[i[1]['Winner/tie']] = int(i[1]['YdsW']) - int(i[1]['YdsL'])
            win_yards.append(0)
        #print(points_dict[i[1]['Winner/tie']])

        
        if i[1]['Loser/tie'] in yards_dict:
            lose_yards.append(yards_dict[i[1]['Loser/tie']])
            yards_dict[i[1]['Loser/tie']] = yards_dict[i[1]['Loser/tie']] - int(i[1]['YdsW']) + int(i[1]['YdsL'])
        else:
            yards_dict[i[1]['Loser/tie']] = -int(i[1]['YdsW']) + int(i[1]['YdsL'])
            lose_yards.append(0)
        
        #turnovers
        if i[1]['Winner/tie'] in tov_dict:
            win_tov.append(tov_dict[i[1]['Winner/tie']])
            tov_dict[i[1]['Winner/tie']] = tov_dict[i[1]['Winner/tie']] + int(i[1]['TOW']) - int(i[1]['TOL'])
        else:
            tov_dict[i[1]['Winner/tie']] = int(i[1]['TOW']) - int(i[1]['TOL'])
            win_tov.append(0)
        #print(points_dict[i[1]['Winner/tie']])

        
        if i[1]['Loser/tie'] in tov_dict:
            lose_tov.append(tov_dict[i[1]['Loser/tie']])
            tov_dict[i[1]['Loser/tie']] = tov_dict[i[1]['Loser/tie']] - int(i[1]['TOW']) + int(i[1]['TOL'])
        else:
            tov_dict[i[1]['Loser/tie']] = -int(i[1]['TOW']) + int(i[1]['TOL'])
            lose_tov.append(0)

df_table['net_ptsw'] = win_dif
df_table['net_ptsl'] = lose_dif
df_table['net_ydsw'] = win_yards
df_table['net_ydsl'] = lose_yards
df_table['net_tovw'] = win_tov
df_table['net_tovl'] = lose_tov
#colts['Winner/tie', 'Loser/tie', 'PtsW', 'PtsL', 'dif']

df_table.head(50)
df_table.to_csv('stats.csv')

In [8]:
years = ['2007','2008','2009','2010','2011','2012','2013','2014','2015']
seasons = []
cols = [1,3,5,7]

for year in years:
    url = f'https://www.pro-football-reference.com/years/{year}/games.htm'
    data = pd.read_html(url)
    df_table = data[0]
    df_table.drop(df_table.columns[cols],axis=1,inplace=True)
    df_table = df_table[df_table.Week.apply(lambda x: str(x).isnumeric())]
    df_table = df_table.reset_index(drop=True)
    points_dict = {}
    yards_dict = {}
    tov_dict = {}
    win_dif = []
    lose_dif = []
    win_yards = []
    lose_yards = []
    win_tov = []
    lose_tov = []

    for i in df_table.iterrows():
        
        #point difference
        if i[1]['Winner/tie'] in points_dict:
            win_dif.append(points_dict[i[1]['Winner/tie']])
            points_dict[i[1]['Winner/tie']] = points_dict[i[1]['Winner/tie']] + int(i[1]['PtsW']) - int(i[1]['PtsL'])
        else:
            points_dict[i[1]['Winner/tie']] = int(i[1]['PtsW']) - int(i[1]['PtsL'])
            win_dif.append(0)

        
        if i[1]['Loser/tie'] in points_dict:
            lose_dif.append(points_dict[i[1]['Loser/tie']])
            points_dict[i[1]['Loser/tie']] = points_dict[i[1]['Loser/tie']] - int(i[1]['PtsW']) + int(i[1]['PtsL'])
        else:
            points_dict[i[1]['Loser/tie']] = -int(i[1]['PtsW']) + int(i[1]['PtsL'])
            lose_dif.append(0)
            
        #net yards
        if i[1]['Winner/tie'] in yards_dict:
            win_yards.append(yards_dict[i[1]['Winner/tie']])
            yards_dict[i[1]['Winner/tie']] = yards_dict[i[1]['Winner/tie']] + int(i[1]['YdsW']) - int(i[1]['YdsL'])
        else:
            yards_dict[i[1]['Winner/tie']] = int(i[1]['YdsW']) - int(i[1]['YdsL'])
            win_yards.append(0)

        
        if i[1]['Loser/tie'] in yards_dict:
            lose_yards.append(yards_dict[i[1]['Loser/tie']])
            yards_dict[i[1]['Loser/tie']] = yards_dict[i[1]['Loser/tie']] - int(i[1]['YdsW']) + int(i[1]['YdsL'])
        else:
            yards_dict[i[1]['Loser/tie']] = -int(i[1]['YdsW']) + int(i[1]['YdsL'])
            lose_yards.append(0)
        
        #turnovers
        if i[1]['Winner/tie'] in tov_dict:
            win_tov.append(tov_dict[i[1]['Winner/tie']])
            tov_dict[i[1]['Winner/tie']] = tov_dict[i[1]['Winner/tie']] + int(i[1]['TOW']) - int(i[1]['TOL'])
        else:
            tov_dict[i[1]['Winner/tie']] = int(i[1]['TOW']) - int(i[1]['TOL'])
            win_tov.append(0)

        
        if i[1]['Loser/tie'] in tov_dict:
            lose_tov.append(tov_dict[i[1]['Loser/tie']])
            tov_dict[i[1]['Loser/tie']] = tov_dict[i[1]['Loser/tie']] - int(i[1]['TOW']) + int(i[1]['TOL'])
        else:
            tov_dict[i[1]['Loser/tie']] = -int(i[1]['TOW']) + int(i[1]['TOL'])
            lose_tov.append(0)

    df_table['net_ptsw'] = win_dif
    df_table['net_ptsl'] = lose_dif
    df_table['net_ydsw'] = win_yards
    df_table['net_ydsl'] = lose_yards
    df_table['net_tovw'] = win_tov
    df_table['net_tovl'] = lose_tov
    df_table['Year'] = year
    seasons.append(df_table)
print(len(seasons))

9


In [9]:
seasons[1]

,Week,Date,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL,net_ptsw,net_ptsl,net_ydsw,net_ydsl,net_tovw,net_tovl,Year
0,1,September 4,New York Giants,Washington Redskins,16,7,354,1,209,0,0,0,0,0,0,0,2008
1,1,September 7,Philadelphia Eagles,St. Louis Rams,38,3,522,0,166,0,0,0,0,0,0,0,2008
2,1,September 7,New York Jets,Miami Dolphins,20,14,293,1,277,1,0,0,0,0,0,0,2008
3,1,September 7,New England Patriots,Kansas City Chiefs,17,10,338,2,284,1,0,0,0,0,0,0,2008
4,1,September 7,Pittsburgh Steelers,Houston Texans,38,17,305,1,234,3,0,0,0,0,0,0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,17,December 28,Philadelphia Eagles,Dallas Cowboys,44,6,303,1,298,5,89,35,1213,808,1,7,2008
252,17,December 28,San Francisco 49ers,Washington Redskins,27,24,359,1,268,1,-45,-28,-329,590,17,0,2008
253,17,December 28,Baltimore Ravens,Jacksonville Jaguars,27,7,431,0,245,4,121,-45,821,-3,-9,4,2008
254,17,December 28,Miami Dolphins,New York Jets,24,17,319,1,331,4,21,56,277,25,-14,-2,2008


In [10]:
df = pd.concat(seasons)

In [12]:
df = df.reset_index(drop=True)
df

,Week,Date,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL,net_ptsw,net_ptsl,net_ydsw,net_ydsl,net_tovw,net_tovl,Year
0,1,September 6,Indianapolis Colts,New Orleans Saints,41,10,452,1,293,3,0,0,0,0,0,0,2007
1,1,September 9,Carolina Panthers,St. Louis Rams,27,13,385,2,238,2,0,0,0,0,0,0,2007
2,1,September 9,Minnesota Vikings,Atlanta Falcons,24,3,302,1,265,2,0,0,0,0,0,0,2007
3,1,September 9,Washington Redskins,Miami Dolphins,16,13,400,2,273,1,0,0,0,0,0,0,2007
4,1,September 9,Denver Broncos,Buffalo Bills,15,14,470,1,184,1,0,0,0,0,0,0,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,17,January 3,San Francisco 49ers,St. Louis Rams,19,16,458,1,364,0,-152,-47,-1433,-1030,4,-3,2015
2300,17,January 3,Carolina Panthers,Tampa Bay Buccaneers,38,10,366,2,386,3,164,-47,724,548,-19,4,2015
2301,17,January 3,Seattle Seahawks,Arizona Cardinals,36,6,354,0,232,3,116,206,1268,1508,-4,-12,2015
2302,17,January 3,Kansas City Chiefs,Oakland Raiders,23,17,339,2,205,1,112,-34,-104,-325,-15,0,2015


In [13]:
df.to_csv('game_stats.csv')